In [ ]:
from pathlib import Path
import subprocess

REPO_URL = "https://github.com/seoultechpse/fenicsx-colab.git"
ROOT = Path("/content")
REPO_DIR = ROOT / "fenicsx-colab"

subprocess.run(["git", "clone", REPO_URL, str(REPO_DIR)], check=True)

USE_COMPLEX = False  # <--- Set True ONLY if you need complex PETSc
USE_CLEAN = False    # <--- Set True to remove existing environment

opts_str = " ".join(
  [o for c, o in [(USE_COMPLEX, "--complex"), (USE_CLEAN, "--clean")] if c]
)

get_ipython().run_line_magic("run", f"{REPO_DIR / 'setup_fenicsx.py'} {opts_str}")

🔧 FEniCSx Setup Configuration
PETSc type      : real
Clean install   : False

⚠️  Google Drive not mounted — using local cache (/content)

🔧 Installing FEniCSx environment...

🔍 Verifying PETSc type...
✅ Installed: Real PETSc (float64)

✨ Loading FEniCSx Jupyter magic... %%fenicsx registered

✅ FEniCSx setup complete!

Next steps:
  1. Run %%fenicsx --info to verify installation
  2. Use %%fenicsx in cells to run FEniCSx code
  3. Use -np N for parallel execution (e.g., %%fenicsx -np 4)

📌 Note: Real PETSc is installed
   - Recommended for most FEM problems
   - For complex problems, reinstall with --complex


---

In [ ]:
%%fenicsx

"""
UFL Elements Examples
=====================

This file contains practical examples demonstrating the concepts
from the "Introduction to UFL Elements" presentation.

Based on FEniCS Workshop materials.
"""

import basix.ufl
import ufl

print("=" * 70)
print("UFL ELEMENTS EXAMPLES")
print("=" * 70)

# =============================================================================
# Example 1: Basic Domain Definition
# =============================================================================
print("\n" + "=" * 70)
print("Example 1: Creating a Computational Domain")
print("=" * 70)

# Step 1: Define the coordinate element
coordinate_element = basix.ufl.element(
    "Lagrange",    # Element family
    "triangle",    # Cell type
    1,             # Degree
    shape=(2,)     # Physical dimension (2D)
)

print(f"Coordinate element: {coordinate_element}")
print(f"  Family: Lagrange")
print(f"  Cell: triangle")
print(f"  Degree: 1")
print(f"  Shape: (2,) - 2D domain")

# Step 2: Create the mesh/domain object
domain = ufl.Mesh(coordinate_element)
print(f"\nDomain created: {domain}")
print(f"  Cell type: {domain.ufl_cell()}")

# =============================================================================
# Example 2: Different Physical Dimensions
# =============================================================================
print("\n" + "=" * 70)
print("Example 2: 2D Manifold Embedded in 3D")
print("=" * 70)

# 2D triangle in 3D space
coord_element_3d = basix.ufl.element(
    "Lagrange",
    "triangle",
    1,
    shape=(3,)     # 2D manifold in 3D space
)

domain_3d = ufl.Mesh(coord_element_3d)
print(f"3D embedded domain: {domain_3d}")
print(f"  Triangle mesh embedded in 3D space")

# =============================================================================
# Example 3: Supported Cell Types
# =============================================================================
print("\n" + "=" * 70)
print("Example 3: Different Cell Types")
print("=" * 70)

cell_types = ["interval", "triangle", "quadrilateral",
              "tetrahedron", "hexahedron", "prism", "pyramid"]

for cell_type in cell_types:
    try:
        if cell_type == "interval":
            shape = (1,)
        elif cell_type in ["triangle", "quadrilateral"]:
            shape = (2,)
        else:  # 3D cells
            shape = (3,)

        elem = basix.ufl.element("Lagrange", cell_type, 1, shape=shape)
        print(f"  ✓ {cell_type:15s} - Created successfully")
    except Exception as e:
        print(f"  ✗ {cell_type:15s} - Error: {e}")

# =============================================================================
# Example 4: Iso-parametric vs Sub-parametric Elements
# =============================================================================
print("\n" + "=" * 70)
print("Example 4: Element Parametrization Types")
print("=" * 70)

cell = str(domain.ufl_cell())

# Iso-parametric: same degree as mesh
iso_element = basix.ufl.element("Lagrange", cell, 1)
print(f"Iso-parametric element (degree 1): {iso_element}")

# Sub-parametric: higher degree than mesh
sub_element = basix.ufl.element("Lagrange", cell, 3)
print(f"Sub-parametric element (degree 3): {sub_element}")
print(f"  → More DOFs than coordinate element")

# =============================================================================
# Example 5: Tensor and Vector Elements
# =============================================================================
print("\n" + "=" * 70)
print("Example 5: Tensor and Vector-Valued Elements")
print("=" * 70)

# Scalar element
scalar_el = basix.ufl.element("Lagrange", cell, 2)
print(f"Scalar element: {scalar_el}")

# Vector element (for displacement, velocity, etc.)
vector_el = basix.ufl.element("Lagrange", cell, 2, shape=(2,))
print(f"Vector element (2D): {vector_el}")

# 3D vector element
vector_el_3d = basix.ufl.element("Lagrange", cell, 2, shape=(3,))
print(f"Vector element (3D): {vector_el_3d}")

# Tensor element (for stress, strain, etc.)
M, N = 3, 3
tensor_el = basix.ufl.element("Lagrange", cell, 1, shape=(M, N))
print(f"Tensor element ({M}x{N}): {tensor_el}")

# Custom dimensional vector
custom_vector = basix.ufl.element("Lagrange", cell, 3, shape=(7,))
print(f"Custom 7-component vector: {custom_vector}")

# =============================================================================
# Example 6: Vector-Valued Elements (N1curl)
# =============================================================================
print("\n" + "=" * 70)
print("Example 6: Intrinsically Vector-Valued Elements (N1curl)")
print("=" * 70)

# N1curl (Nédélec) element - for electromagnetic problems
curl_el = basix.ufl.element("N1curl", cell, 1)
print(f"N1curl element: {curl_el}")
print(f"  → Used for Maxwell equations")
print(f"  → DO NOT use 'shape' parameter for these!")

# Creating multiple N1curl spaces
blocked_curl = basix.ufl.mixed_element([curl_el for _ in range(4)])
print(f"\nBlocked N1curl (4 spaces): {blocked_curl}")

# =============================================================================
# Example 7: Element Enrichment
# =============================================================================
print("\n" + "=" * 70)
print("Example 7: Element Enrichment (Adding Bubble Functions)")
print("=" * 70)

# Enriched element: Lagrange + Bubble
enriched_element = basix.ufl.enriched_element([
    basix.ufl.element("Lagrange", cell, 2),
    basix.ufl.element("Bubble", cell, 3)
])
print(f"Enriched element (P2 + Bubble): {enriched_element}")
print(f"  Linear Lagrange spans: {{1, x, y}}")
print(f"  Bubble adds: xy(1-x-y)")
print(f"  Combined: {{1, x, y, xy(1-x-y)}}")

# =============================================================================
# Example 8: MINI Element for Stokes
# =============================================================================
print("\n" + "=" * 70)
print("Example 8: MINI Element (Stokes Problem)")
print("=" * 70)

# Velocity: enriched element blocked to vector
el_u = basix.ufl.blocked_element(enriched_element, shape=(2,))
print(f"Velocity element (enriched, blocked): {el_u}")

# Pressure: linear Lagrange
el_p = basix.ufl.element("Lagrange", cell, 1)
print(f"Pressure element (P1): {el_p}")

# Mixed element for Stokes
mini_element = basix.ufl.mixed_element([el_u, el_p])
print(f"\nMINI element (velocity + pressure): {mini_element}")
print(f"  → Stable for Stokes equations")
print(f"  → Satisfies inf-sup condition")

# =============================================================================
# Example 9: Taylor-Hood Element for Stokes
# =============================================================================
print("\n" + "=" * 70)
print("Example 9: Taylor-Hood Element (Stokes Problem)")
print("=" * 70)

# Velocity: P2 vector
V_el = basix.ufl.element("Lagrange", cell, 2, shape=(2,))
print(f"Velocity element (P2 vector): {V_el}")

# Pressure: P1 scalar
Q_el = basix.ufl.element("Lagrange", cell, 1)
print(f"Pressure element (P1 scalar): {Q_el}")

# Taylor-Hood mixed element
taylor_hood = basix.ufl.mixed_element([V_el, Q_el])
print(f"\nTaylor-Hood element: {taylor_hood}")
print(f"  → Higher accuracy than MINI")
print(f"  → More DOFs than MINI")
print(f"  → Also satisfies inf-sup condition")

# =============================================================================
# Example 10: Discontinuous Galerkin (DG) Elements
# =============================================================================
print("\n" + "=" * 70)
print("Example 10: Discontinuous Galerkin (DG) Elements")
print("=" * 70)

# Method 1: Using discontinuous flag
dg_el_1 = basix.ufl.element("Lagrange", cell, 1, discontinuous=True)
print(f"DG element (method 1): {dg_el_1}")

# Method 2: Using "DG" family name (shorthand)
dg_el_2 = basix.ufl.element("DG", cell, 1)
print(f"DG element (method 2): {dg_el_2}")

# Verify they are the same
print(f"\nAre they equal? {dg_el_1 == dg_el_2}")

# Higher order DG
dg_el_p3 = basix.ufl.element("DG", cell, 3)
print(f"Higher order DG (P3): {dg_el_p3}")

# =============================================================================
# Example 11: Practical Application - Complete Stokes Setup
# =============================================================================
print("\n" + "=" * 70)
print("Example 11: Complete Stokes Problem Setup")
print("=" * 70)

# 1. Define domain
coord_el = basix.ufl.element("Lagrange", "triangle", 1, shape=(2,))
domain_stokes = ufl.Mesh(coord_el)
cell_stokes = str(domain_stokes.ufl_cell())

print("Step 1: Domain created")
print(f"  Coordinate element: P1 Lagrange triangles")

# 2. Define Taylor-Hood element
V_stokes = basix.ufl.element("Lagrange", cell_stokes, 2, shape=(2,))
Q_stokes = basix.ufl.element("Lagrange", cell_stokes, 1)
W_stokes = basix.ufl.mixed_element([V_stokes, Q_stokes])

print("\nStep 2: Taylor-Hood element defined")
print(f"  Velocity: P2 vector (2D)")
print(f"  Pressure: P1 scalar")

# This would be followed by creating function spaces with DOLFINx
# (not shown here as we're focusing on UFL elements)
print("\nStep 3: Next step would be creating function spaces with DOLFINx")
print("  Example: V = dolfinx.fem.functionspace(domain_stokes, W_stokes)")

# =============================================================================
# Example 12: Comparison of Different Element Families
# =============================================================================
print("\n" + "=" * 70)
print("Example 12: Different Element Families")
print("=" * 70)

families = {
    "Lagrange": "Standard continuous elements",
    "DG": "Discontinuous Galerkin",
    "N1curl": "Nédélec (edge elements)",
    "RT": "Raviart-Thomas (face elements)",
    "Bubble": "Bubble functions"
}

for family, description in families.items():
    try:
        if family in ["N1curl", "RT"]:
            # These are vector-valued by nature
            elem = basix.ufl.element(family, cell, 1)
        else:
            elem = basix.ufl.element(family, cell, 1)
        print(f"  ✓ {family:12s}: {description}")
    except Exception as e:
        print(f"  ✗ {family:12s}: {description} - {str(e)[:40]}")

# =============================================================================
# Example 13: Element Properties
# =============================================================================
print("\n" + "=" * 70)
print("Example 13: Inspecting Element Properties")
print("=" * 70)

test_element = basix.ufl.element("Lagrange", "triangle", 2, shape=(2,))
print(f"Element: {test_element}")
print(f"  Family: Lagrange")
print(f"  Cell: triangle")
print(f"  Degree: 2")
print(f"  Value shape: (2,)")
print(f"  Use case: 2D displacement/velocity fields")

# =============================================================================
# Summary
# =============================================================================
print("\n" + "=" * 70)
print("SUMMARY")
print("=" * 70)
print("""
Key Concepts Demonstrated:
1. Creating domains with coordinate elements
2. Iso-parametric vs sub-parametric elements
3. Scalar, vector, and tensor elements
4. Vector-valued elements (N1curl, RT)
5. Element enrichment (Lagrange + Bubble)
6. Mixed elements (MINI, Taylor-Hood)
7. Discontinuous Galerkin (DG) elements
8. Complete problem setup (Stokes)

Next Steps:
- Use these elements with DOLFINx to create function spaces
- Define variational forms
- Apply boundary conditions
- Solve actual PDEs
""")

print("\n" + "=" * 70)
print("Examples completed successfully!")
print("=" * 70)

UFL ELEMENTS EXAMPLES

Example 1: Creating a Computational Domain
Coordinate element: blocked element (Basix element (P, triangle, 1, gll_warped, unset, False, float64, []), (2,))
  Family: Lagrange
  Cell: triangle
  Degree: 1
  Shape: (2,) - 2D domain

Domain created: <Mesh #0>
  Cell type: triangle

Example 2: 2D Manifold Embedded in 3D
3D embedded domain: <Mesh #1>
  Triangle mesh embedded in 3D space

Example 3: Different Cell Types
  ✓ interval        - Created successfully
  ✓ triangle        - Created successfully
  ✓ quadrilateral   - Created successfully
  ✓ tetrahedron     - Created successfully
  ✓ hexahedron      - Created successfully
  ✓ prism           - Created successfully
  ✓ pyramid         - Created successfully

Example 4: Element Parametrization Types
Iso-parametric element (degree 1): Basix element (P, triangle, 1, gll_warped, unset, False, float64, [])
Sub-parametric element (degree 3): Basix element (P, triangle, 3, gll_warped, unset, False, float64, [])
  → M